In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
hou_csv= "Data/HOU_season_stats.csv"
wsn_csv = "Data/WSN_season_stats.csv"
team_batting_csv = "Data/TeamBatting.csv"
team_fielding_csv = "Data/TeamFielding.csv"
team_pitching_csv = "Data/TeamPitching.csv"
team_waa_csv = "Data/TeamWAA.csv"

In [3]:
astros = pd.read_csv(hou_csv)
nats = pd.read_csv(wsn_csv)
team_batting = pd.read_csv(team_batting_csv)
team_fielding = pd.read_csv(team_fielding_csv)
team_pitching = pd.read_csv(team_pitching_csv)
team_waa = pd.read_csv(team_waa_csv)

In [4]:
astros_nats = pd.concat([astros, nats])
astros_nats.head()

,Gm#,Tm,Opp,W/L,R,RA
0,1,HOU,TBR,W,5,1
1,2,HOU,TBR,L,2,4
2,3,HOU,TBR,L,1,3
3,4,HOU,TBR,L,1,3
4,5,HOU,TEX,W,2,1


In [5]:
combined_stats1 = pd.merge(team_batting, team_fielding,  on = "Tm", how = "inner")
combined_stats1.head()

,Tm,#Bat,BatAge,R/G,G_x,PA,AB,R,H,2B,...,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr
0,ARI,45,28.7,5.02,162,6315,5633,813,1419,288,...,6041,4395,1560,86,136,0.986,53,5,75.0,7.0
1,ATL,50,28.0,5.28,162,6302,5560,855,1432,277,...,5952,4352,1522,78,154,0.987,5,0,37.0,3.0
2,BAL,58,26.5,4.50,162,6189,5596,729,1379,252,...,5879,4329,1442,108,155,0.982,6,1,-89.0,-8.0
3,BOS,47,27.3,5.56,162,6475,5770,901,1554,345,...,5937,4413,1436,88,115,0.985,-7,-1,-37.0,-3.0
4,CHC,52,27.7,5.02,162,6195,5461,814,1378,270,...,6063,4326,1619,118,141,0.981,9,1,-7.0,-1.0


In [6]:
combined_stats =pd.merge(combined_stats1, team_pitching,  on = "Tm", how = "inner")
combined_stats.head()

,Tm,#Bat,BatAge,R/G,G_x,PA,AB,R_x,H_x,2B,...,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,LOB_y
0,ARI,45,28.7,5.02,162,6315,5633,813,1419,288,...,6230,105,4.40,1.308,8.6,1.4,3.2,8.8,2.77,1092
1,ATL,50,28.0,5.28,162,6302,5560,855,1432,277,...,6243,109,4.39,1.357,8.8,1.3,3.4,8.6,2.54,1148
2,BAL,58,26.5,4.50,162,6189,5596,729,1379,252,...,6396,84,5.56,1.459,9.6,1.9,3.5,7.8,2.22,1086
3,BOS,47,27.3,5.56,162,6475,5770,901,1554,345,...,6400,103,4.28,1.379,8.7,1.3,3.7,10.0,2.70,1159
4,CHC,52,27.7,5.02,162,6195,5461,814,1378,270,...,6190,109,4.25,1.325,8.6,1.2,3.3,9.0,2.70,1147


In [7]:
astros_nats_stats = pd.merge(astros_nats, combined_stats, on = "Tm", how = "inner")
astros_nats_stats.head()

,Gm#,Tm,Opp,W/L,R,RA,#Bat,BatAge,R/G,G_x,...,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,LOB_y
0,1,HOU,TBR,W,5,1,45,29.0,5.68,162,...,5995,127,3.98,1.13,7.4,1.4,2.8,10.3,3.73,968
1,2,HOU,TBR,L,2,4,45,29.0,5.68,162,...,5995,127,3.98,1.13,7.4,1.4,2.8,10.3,3.73,968
2,3,HOU,TBR,L,1,3,45,29.0,5.68,162,...,5995,127,3.98,1.13,7.4,1.4,2.8,10.3,3.73,968
3,4,HOU,TBR,L,1,3,45,29.0,5.68,162,...,5995,127,3.98,1.13,7.4,1.4,2.8,10.3,3.73,968
4,5,HOU,TEX,W,2,1,45,29.0,5.68,162,...,5995,127,3.98,1.13,7.4,1.4,2.8,10.3,3.73,968


In [30]:
# separate w/l to use to train
yTr = astros_nats_stats['W/L']
xTr = astros_nats_stats.drop(columns =['W/L','Tm', 'Opp','R','RA','Gm#','W-L%','#P'])
# xTr2 = astros_nats_stats[['R/G','OBP','OPS','TB','RA/G','DefEff','Fld%','ERA','ERA+','WHIP','H9','HR9','BB9','SO9','LOB']]
xTr.head()

,#Bat,BatAge,R/G,G_x,PA,AB,R_x,H_x,2B,3B,...,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,LOB_y
0,45,29.0,5.68,162,6394,5613,920,1538,323,28,...,5995,127,3.98,1.13,7.4,1.4,2.8,10.3,3.73,968
1,45,29.0,5.68,162,6394,5613,920,1538,323,28,...,5995,127,3.98,1.13,7.4,1.4,2.8,10.3,3.73,968
2,45,29.0,5.68,162,6394,5613,920,1538,323,28,...,5995,127,3.98,1.13,7.4,1.4,2.8,10.3,3.73,968
3,45,29.0,5.68,162,6394,5613,920,1538,323,28,...,5995,127,3.98,1.13,7.4,1.4,2.8,10.3,3.73,968
4,45,29.0,5.68,162,6394,5613,920,1538,323,28,...,5995,127,3.98,1.13,7.4,1.4,2.8,10.3,3.73,968


In [31]:
y=yTr


In [32]:
y = y.replace('L-wo', 'L')
y.head()

0    W
1    L
2    L
3    L
4    W
Name: W/L, dtype: object

In [33]:
# Fills Outcome column based on wins vs losses
y2 =pd.get_dummies(y)
y2.head()

,L,W,W-wo
0,0,1,0
1,1,0,0
2,1,0,0
3,1,0,0
4,0,1,0


In [34]:
y2 =y2.drop(columns =['L'], axis = 1)
y2.head()

,W,W-wo
0,1,0
1,0,0
2,0,0
3,0,0
4,1,0


In [35]:
yTr2 = y2.values
yTr2.shape

(324, 2)

In [36]:
X = xTr
feature_names = X.columns
y = yTr2
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [37]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [38]:
rf.score(X_train, y_train)

0.5925925925925926

In [39]:
rf.score(X_test, y_test)

0.5925925925925926

In [40]:
sorted(zip(rf.feature_importances_, feature_names), reverse = True)

[(0.04, 'PO'),
 (0.03, 'Rtot'),
 (0.03, 'R_y'),
 (0.03, 'PA'),
 (0.03, 'GDP'),
 (0.03, 'AB'),
 (0.025, 'W'),
 (0.025, 'SO_y'),
 (0.025, 'SO9'),
 (0.025, 'R/G'),
 (0.025, 'OBP'),
 (0.025, 'LOB_x'),
 (0.025, 'IP'),
 (0.02, 'SLG'),
 (0.02, 'SH'),
 (0.02, 'RBI'),
 (0.02, 'RA/G_y'),
 (0.02, 'LOB_y'),
 (0.02, 'HR_x'),
 (0.02, 'Fld%'),
 (0.02, 'ERA+'),
 (0.02, 'DefEff'),
 (0.02, 'CG_y'),
 (0.02, 'CG_x'),
 (0.02, 'BB_x'),
 (0.02, '#Bat'),
 (0.015, 'SO/W'),
 (0.015, 'PAge'),
 (0.015, 'OPS'),
 (0.015, 'L'),
 (0.015, 'Inn'),
 (0.015, 'FIP'),
 (0.015, 'ERA'),
 (0.015, 'Ch'),
 (0.015, 'BatAge'),
 (0.015, 'BK'),
 (0.015, 'BB9'),
 (0.015, 'BA'),
 (0.015, '3B'),
 (0.01, 'WP'),
 (0.01, 'SV'),
 (0.01, 'RA/G_x'),
 (0.01, 'OPS+'),
 (0.01, 'IBB_y'),
 (0.01, 'IBB_x'),
 (0.01, 'HR9'),
 (0.01, 'GF'),
 (0.01, 'BF'),
 (0.01, '2B'),
 (0.005, 'WHIP'),
 (0.005, 'TB'),
 (0.005, 'SB'),
 (0.005, 'Rtot/yr'),
 (0.005, 'Rdrs'),
 (0.005, 'R_x'),
 (0.005, 'H_y'),
 (0.005, 'H_x'),
 (0.005, 'HR_y'),
 (0.005, 'HBP_y'),
 (0.0

In [41]:
import pickle

In [42]:
ws_model = pickle.dumps(rf)

In [43]:
from sklearn.externals import joblib

In [44]:
joblib.dump(ws_model, 'ws.pkl')

['ws.pkl']

In [45]:
rf.predict(X_test)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.

In [46]:
rf.predict_proba(X_test)

[array([[0.37807183, 0.62192817],
        [0.37807183, 0.62192817],
        [0.37807183, 0.62192817],
        [0.37807183, 0.62192817],
        [0.37807183, 0.62192817],
        [0.43870728, 0.56129272],
        [0.43870728, 0.56129272],
        [0.43870728, 0.56129272],
        [0.43870728, 0.56129272],
        [0.43870728, 0.56129272],
        [0.37807183, 0.62192817],
        [0.43870728, 0.56129272],
        [0.43870728, 0.56129272],
        [0.43870728, 0.56129272],
        [0.43870728, 0.56129272],
        [0.43870728, 0.56129272],
        [0.43870728, 0.56129272],
        [0.43870728, 0.56129272],
        [0.43870728, 0.56129272],
        [0.43870728, 0.56129272],
        [0.37807183, 0.62192817],
        [0.43870728, 0.56129272],
        [0.37807183, 0.62192817],
        [0.37807183, 0.62192817],
        [0.37807183, 0.62192817],
        [0.37807183, 0.62192817],
        [0.37807183, 0.62192817],
        [0.37807183, 0.62192817],
        [0.43870728, 0.56129272],
        [0.378